In [13]:
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.data import DatasetCatalog, MetadataCatalog
from detectron2.config import get_cfg
from detectron2.engine import default_argument_parser, default_setup, launch
from typing import Dict, Set, List, Tuple, Iterator
import os
import logging
import json
import copy

# hacky way to register
from ubteacher.modeling import *
from ubteacher.engine import *
from ubteacher.engine.trainer import UBTeacherTrainer, UBRCNNTeacherTrainer, BaselineTrainer
from ubteacher import add_ubteacher_config
from ubteacher.utils.utils import (TrainHelper, 
                    get_categorical_map, 
                    get_annotypes_for_dataset)

In [16]:
train_sample = '/mnt/RSX/Datasets_pathology/SRI_OSCC_lymph_train'

In [51]:
def find_anno_dir(parent_dir: str) -> List[str]:
    """
    Find qupath exported annotations directory
    """
    
    if os.path.exists(os.path.join(parent_dir, 'xupath_annotations_latest')):
        return os.path.join(parent_dir, 'qupath_annotations_latest')
    else:
        anno_dirs = []
        for root, dirs, files in os.walk(parent_dir):
            for d in dirs:
                if 'annotations' in d:
                    anno_dirs.append(os.path.join(root, d))
        # user chooses if there are multiple annotation folders
        print('Found multiple annotation folders:')
        for i, anno_dir in enumerate(anno_dirs):
            print(f'{i}: {os.path.relpath(anno_dir, parent_dir)}')
        choice = input('Choose annotation folder index')
        if choice.isdigit() and int(choice) < len(anno_dirs):
            return anno_dirs[int(choice)]    
        else:
            raise ValueError('Annotation folder not found')

In [52]:
find_anno_dir(train_sample)

Found multiple annotation folders:
0: qupath_annotations_latest
1: qupath_annotations_V4
2: qupath_annotations_V3
3: qupath_annotations_V2
4: qupath_annotations_V1


'/mnt/RSX/Datasets_pathology/SRI_OSCC_lymph_train/qupath_annotations_latest'